In [15]:
import pandas as pd
import numpy as np
import re
import spacy

In [16]:
df = pd.read_csv('KPMG Tax Case - Scraped Data.csv')

# Remove the German Translations

df = df[df["Text"].str.contains("Duitse vertaling")==False]

# Remove empty text rows.

df.dropna(axis = 0, how ='any', inplace = True)

df['Cleaned Text'] = ''
df['Summary_1'] = ''
df['Summary_2'] = ''

df.head()

,Date,Title,Numac,Link FR,Link NL,Text,Cleaned Text,Summary_1,Summary_2
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,,,
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\n\neinde eerste woord laatste woord\nPub...,,,
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,,,
5,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020020094,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,,,
6,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\nAdministratio...,2020010193,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\n\neinde eerste woord laatste woord\nPub...,,,


In [36]:
df.shape

(662, 9)

In [37]:
# Test text

for idx, row in df.loc[100:663].iterrows():
    string = df.loc[idx,'Text']
    # Removing the first common part

    splitted_text = string.split('Numac')
    splitted_text = splitted_text[1]

    # testing

    trial = splitted_text.replace('\n',' ')
    trial = trial.replace('/','')
    trial = trial.replace('§','')
    
    # result = ''.join([i for i in trial if not i.isdigit()])
    
    
    # Creating Tokens

    nlp = spacy.load('nl_core_news_sm')
    text = trial
    tokenized = nlp(text)
    tokens = [token.text for token in tokenized]

    stopwords = spacy.lang.nl.stop_words.STOP_WORDS

    text_no_stop = [lemma for lemma in tokens if lemma not in stopwords]
    cleaned = ' '.join(text_no_stop)

    index = cleaned.rfind('begin')
    
    corpus = trial[:index]
    
    df.loc[idx,'Cleaned Text'] = corpus


IndexError: list index out of range

In [46]:
corpus

' : 2022031428 FEDERALE OVERHEIDSDIENST SOCIALE ZEKERHEID 23 MAART 2022. - Koninklijk besluit tot wijziging van het koninklijk besluit nr. 21 van 14 mei 2020 houdende tijdelijke aanpassingen aan de vergoedingsvoorwaarden en administratieve regels in de verplichte verzekering voor geneeskundige verzorging ten gevolge van de COVID-19 pandemie   FILIP, Koning der Belgen, Aan allen die nu zijn en hierna wezen zullen, Onze Groet. Gelet op de wet van 27 maart 2020 die machtiging verleent aan de Koning om maatregelen te nemen in de strijd tegen de verspreiding van het coronavirus COVID-19 (II), de artikelen 5,  1, 1°, en 6° ; Gelet op het koninklijk besluit nr. 21 van 14 mei 2020 houdende tijdelijke aanpassingen aan de vergoedingsvoorwaarden en administratieve regels in de verplichte verzekering voor geneeskundige verzorging ten gevolge van de COVID-19 pandemie, artikel 31, bekrachtigd en gewijzigd bij de wet van 24 december 2020 tot bekrachtiging van de koninklijke besluiten genomen met toep

In [48]:
# mBart finetune Model

import transformers
undisputed_best_model = transformers.MBartForConditionalGeneration.from_pretrained(
    "ml6team/mbart-large-cc25-cnn-dailymail-nl-finetune"
)
tokenizer = transformers.MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")
summarization_pipeline = transformers.pipeline(
    task="summarization",
    model=undisputed_best_model,
    tokenizer=tokenizer,
)
summarization_pipeline.model.config.decoder_start_token_id = tokenizer.lang_code_to_id[
    "nl_XX"
]


for idx, row in df.loc[100:300].iterrows():
    
    article = df.loc[idx,'Cleaned Text']  
    df.loc[idx,'Summary_1'] = summarization_pipeline(
        article,
        do_sample=True,
        top_p=0.75,
        top_k=50,
        # num_beams=4,
        min_length=50,
        early_stopping=True,
        truncation=True,
    )[0]["summary_text"]

Your max_length is set to 1024, but you input_length is only 440. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=220)
Your max_length is set to 1024, but you input_length is only 647. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=323)
Your max_length is set to 1024, but you input_length is only 918. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=459)
Your max_length is set to 1024, but you input_length is only 509. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=254)
Your max_length is set to 1024, but you input_length is only 602. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=301)
Your max_length is set to 1024, but you input_length is only 331. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=165)
Your max_length is set to 1024, but you input_length is only 313

Your max_length is set to 1024, but you input_length is only 426. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=213)
Your max_length is set to 1024, but you input_length is only 317. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=158)
Your max_length is set to 1024, but you input_length is only 541. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=270)
Your max_length is set to 1024, but you input_length is only 468. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=234)
Your max_length is set to 1024, but you input_length is only 103. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)
Your max_length is set to 1024, but you input_length is only 139. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=69)
Your max_length is set to 1024, but you input_length is only 816. 

In [ ]:
'''# mBart finetune Model

import transformers
undisputed_best_model = transformers.MBartForConditionalGeneration.from_pretrained(
    "ml6team/mbart-large-cc25-cnn-dailymail-nl-finetune"
)
tokenizer = transformers.MBartTokenizer.from_pretrained("facebook/mbart-large-cc25")
summarization_pipeline = transformers.pipeline(
    task="summarization",
    model=undisputed_best_model,
    tokenizer=tokenizer,
)
summarization_pipeline.model.config.decoder_start_token_id = tokenizer.lang_code_to_id[
    "nl_XX"
]
article = corpus  # Dutch
summarization_pipeline(
    article,
    do_sample=True,
    top_p=0.75,
    top_k=50,
    # num_beams=4,
    min_length=50,
    early_stopping=True,
    truncation=True,
)[0]["summary_text"]'''

In [49]:
df = df.reset_index(drop=True)

In [50]:
df['Summary_1'][250]

'De Belgische overheid heeft wijzigingen aangebracht in het Fiscaliteitsbericht in verband met de investeringsaftrek. Volgens het Ministerie van Financiën moet de aftrek van de belastingen tussen 2019 en 2020 als volgt worden verlengd:'

In [26]:
df.head(100)

,Date,Title,Numac,Link FR,Link NL,Text,Cleaned Text,Summary_1,Summary_2
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\nREGION DE BRUXEL...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020010053 BRUSSELS HOOFDSTEDELIJK GEWEST 8...,"Demissionair minister van Financiën, belast me...",
1,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 DECEM...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\n\neinde eerste woord laatste woord\nPub...,: 2020010214 MINISTERIE VAN DE FRANSE GEMEENS...,De Franse Gemeenschap en de Executieve van de ...,
2,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020040138 FEDERALE OVERHEIDSDIENST FINANCI...,De ministers van Financiën en de staatssecreta...,
3,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\n20 JANVIER 20...,2020020094,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020020094 FEDERALE OVERHEIDSDIENST FINANCI...,Het ministerie van Financiën heeft het ministe...,
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\nAdministratio...,2020010193,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\n\neinde eerste woord laatste woord\nPub...,: 2020010193 FEDERALE OVERHEIDSDIENST FINANCI...,,
...,...,...,...,...,...,...,...,...,...
95,5/29/2020,FR\n\nNL\n\nDE\n\n Image du sommaire du 2020-0...,2020041431,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020041431 FEDERALE OVERHEIDSDIENST JUSTITI...,Het Gerechtelijk Wetboek is van toepassing op ...,
96,5/29/2020,SERVICE PUBLIC FEDERAL SECURITE SOCIALE\n20 MA...,2020202428,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020202428 FEDERALE OVERHEIDSDIENST SOCIALE...,Het koninklijk besluit tot wijziging van het k...,
97,5/29/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 MAI 2...,2020021013,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020021013 MINISTERIE VAN DE FRANSE GEMEENS...,De Franse Gemeenschap heeft een besluit genome...,
98,5/29/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\n20 MAI 2...,2020021022,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article_b...,NL FR\nbelgiëlex.be - Kruispuntbank Wetgevi...,: 2020021022 MINISTERIE VAN DE FRANSE GEMEENS...,"De Gemeenschapsraad voor preventie, hulpverlen...",


In [47]:
df['Summary_1'][4]

''

In [51]:
df.to_excel("KPMG Tax Case - Summarized.xlsx", index=False)
df.to_csv("KPMG Tax Case - CSV_Summarized.csv", index=False, encoding="utf-8-sig")

PermissionError: [Errno 13] Permission denied: 'KPMG Tax Case - CSV_Summarized.csv'